In [1]:
import numpy as np
import pandas as pd
import gudhi as gd
import sys
import matplotlib.pyplot as plt
import networkx as nx
from ripser import Rips
import pickle
import os
import persim
from sklearn.manifold import TSNE
from matplotlib.pyplot import figure

In [2]:
def sub_vis(s_n, opt = "debug"):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(5,10))
    sub_g = nx.ego_graph(G, s_n, radius=5)
    sub_g.remove_edges_from(list(nx.selfloop_edges(sub_g)))
    edgs = list(sub_g.edges)
    col = []
    row = []
    for edg in edgs:
        col.append(edg[0])
        row.append(edg[1])

    p_input = set()
    p_output = set()

    p_input.add(s_n)

    o_d = sub_g.out_degree
    i_d = sub_g.in_degree

    for node in sub_g:
        if o_d[node] == 0 and i_d[node] == 0:
            continue

        if o_d[node] == 0:
            p_output.add(node)

        if i_d[node] == 0:
            p_input.add(node)
    
    
    
    
    gen = nx.dfs_edges(sub_g, source=s_n)
    delay_dict = {}
    delay_dict[s_n] = 0
    for edge in list(gen):
        n1 = edge[0]
        n2 = edge[1]
        delay = delay_dict[n1] + 1
        if n2 in delay_dict.keys():
            if delay > delay_dict[n2]:
                delay_dict[n2] = delay
                    
        else:
            delay_dict[n2] = delay
    
    

    d_lst = []
    for delay in range(max(delay_dict.values()) + 1):
        delay_level_lst = []
        for node in delay_dict.keys():
            if delay_dict[node] == delay:
                delay_level_lst.append(node)

        if len(delay_level_lst) == 0:
            continue
        d_lst.append(delay_level_lst)

    to_level = len(d_lst) 
    mid = np.max([len(lst) for lst in d_lst])/2

    pos = {}

    for delay in range(len(d_lst)):
        lst = d_lst[delay]
        for in_pos in range(len(lst)):
            node = lst[in_pos]
            pos[node] = ((mid - len(lst)) + in_pos*2, (len(d_lst) - delay))

    ax = axes[0]
    dgms = np.load(f"dgms/pd_{s_n}.npy", allow_pickle=True)
    ax.scatter([dgms[0][i][1][0] for i in range(len(dgms[0]))], [dgms[0][i][1][1] for i in range(len(dgms[0]))], color="blue")
    ax.scatter([dgms[2][i][1][0] for i in range(len(dgms[2]))], [dgms[2][i][1][1] for i in range(len(dgms[2]))], color="blue")
    ax.scatter([dgms[1][i][1][0] for i in range(len(dgms[1]))], [dgms[1][i][1][1] for i in range(len(dgms[1]))], color="blue")
    ax.scatter([dgms[3][i][1][0] for i in range(len(dgms[3]))], [dgms[3][i][1][1] for i in range(len(dgms[3]))], color="orange")
    ax.plot([0, to_level],[0, to_level])
    ax = axes[1]

    if opt=="debug":
        color = []
        for node in sub_g:
            if node in p_input:
                color.append("red")
            else:
                color.append("blue")
        print(delay_dict)
        nx.draw(sub_g, ax=ax, node_color=color, labels={node:node for node in sub_g})
    
    if opt=="show":
        color = []
        for node in pos.keys():
            if node in p_input:
                color.append("red")
            else:
                color.append("blue")
        
        print(delay_dict)
        nx.draw(sub_g, ax=ax, pos=pos, node_color=color, labels={node:node for node in pos.keys()}, nodelist=pos.keys())
    
    

In [7]:
class Graph:
    # Constructor
    def __init__(self, edges, n):
 
        # A list of lists to represent an adjacency list
        self.adjList = [[] for _ in range(n)]
 
        # add edges to the directed graph
        for (source, dest, weight) in edges:
            self.adjList[source].append((dest, weight))
 
 
# Perform DFS on the graph and set the departure time of all
# vertices of the graph
def DFS(graph, v, discovered, departure, time):
    # mark the current node as discovered
    discovered[v] = True
 
    # set arrival time – not needed
    # time = time + 1
 
    # do for every edge (v, u)
    for (u, w) in graph.adjList[v]:
        # if `u` is not yet discovered
        if not discovered[u]:
            time = DFS(graph, u, discovered, departure, time)
 
    # ready to backtrack
    # set departure time of vertex `v`
    departure[time] = v
    time = time + 1
 
    return time
 
def findLongestDistance(graph, source, n):
    # `departure` stores vertex number having its departure
    # time equal to the index of it
    departure = [-1] * n
 
    # to keep track of whether a vertex is discovered or not
    discovered = [False] * n
    time = 0
 
    # perform DFS on all undiscovered vertices
    for i in range(n):
        if not discovered[i]:
            time = DFS(graph, i, discovered, departure, time)
 
    cost = [sys.maxsize] * n
    cost[source] = 0
 
    # Process the vertices in topological order, i.e., in order
    # of their decreasing departure time in DFS
    added = []
    
    for i in reversed(range(n)):
     
        # for each vertex in topological order,
        # relax the cost of its adjacent vertices
        v = departure[i]
        # edge from `v` to `u` having weight `w`
        for (u, w) in graph.adjList[v]:
            w = -w     # make edge weight negative
            # if the distance to destination `u` can be shortened by
            # taking edge (v, u), then update cost to the new lower value
            if cost[v] != sys.maxsize and cost[v] + w < cost[u]:
                cost[u] = cost[v] + w
                
            
    
    
    dist = dict()
    for i in range(n):
        dist[i] = {-cost[i]}
        
    return dist

In [9]:
netlist_name = "jpeg_300_off_low_GENUS"
G = nx.read_gpickle(f"/home/zluo/input/{netlist_name}.gpickle")

p_input = set()
p_output = set()
o_d = G.out_degree
i_d = G.in_degree

for node in G:
    if o_d[node] == 0 and i_d[node] == 0:
        continue

    if o_d[node] == 0:
        p_output.add(node)

    if i_d[node] == 0:
        p_input.add(node)


all_raw = []

for v in p_input:
    all_raw.append(nx.shortest_path_length(G, v))    

delay_dict = all_raw[0]
delay_dict = {key: [delay_dict[key]] for key in delay_dict.keys()}
for dist in all_raw[1:]:
    for key in dist.keys(): 
        val = dist[key]
        if key in delay_dict.keys():
            delay_dict[key] = delay_dict[key] + [val]
        else:
            delay_dict[key] = [val]

delay_dict = {key: min(delay_dict[key]) for key in delay_dict.keys()}

with open(f'/home/zluo/output/delay_dict_{netlist_name}_shortest.pkl', 'wb') as f:
    pickle.dump(delay_dict, f)

In [4]:
#read in the data

col_row = np.load(f"/home/zluo/output/{netlist_name}.npy")

with open(f'/home/zluo/output/{netlist_name}bTerm.pkl', 'rb') as handle:
    dterm = pickle.load(handle)
    dterm = dict((v,k) for k,v in dterm.items())
    
with open(f'/home/zluo/output/{netlist_name}inst.pkl', 'rb') as handle:
    dinst = pickle.load(handle)
    dinst = dict((v,k) for k,v in dinst.items())

with open(f'/home/zluo/output/{netlist_name}all.pkl', 'rb') as handle:
    dall = pickle.load(handle)

    # dall = {}
# dall.update(dterm)
# dall.update(dinst)

# dtype = {}
# for node in dall.keys():
#     if node in dterm:
#         dtype[node] = "bTerm"
#     else:
#         dtype[node] = "inst"

In [13]:
netlist_name = "jpeg_300_off_low_GENUS_o"
col_row = np.load(f"{netlist_name}.npy")
col = col_row[0]
row = col_row[1]
edgs = np.array([col, row]).T

In [14]:
# Building the Graph (undirected)
edgelist = []
for edge in col_row.T:
    new_edge = (edge[0], edge[1])
    edgelist.append(new_edge)
    
G = nx.DiGraph(edgelist)
nodelist = [node for node in G.nodes]
nodelist = sorted(nodelist)
adj = nx.adjacency_matrix(G, nodelist=nodelist)
p_input = set()
p_output = set()
o_d = G.out_degree
i_d = G.in_degree

for node in G:
    if o_d[node] == 0 and i_d[node] == 0:
        continue

    if o_d[node] == 0:
        p_output.add(node)

    if i_d[node] == 0:
        p_input.add(node)

In [16]:
#####################################
all_dist_raw = []
edgs_d = [(lst[0], lst[1], 1) for lst in edgs] 
n = np.max(list(col) + list(row)) + 1
graph = Graph(edgs_d, n)
for node in p_input: all_dist_raw.append(findLongestDistance(graph, node, n))

# building the delay based dictionary
delay_dict = all_dist_raw[0]
delay_dict = {key: list(delay_dict[key]) for key in delay_dict.keys()}
for dist in all_dist_raw[1:]:
    for key in dist.keys(): 
        val = dist[key]
        if key in delay_dict.keys():
            delay_dict[key] = delay_dict[key] + list(val)
        else:
            delay_dict[key] = list(val)

delay_dict = {key: max(delay_dict[key]) for key in delay_dict.keys()}

with open(f'delay_dict_{netlist_name}.pkl', 'wb') as f:
    pickle.dump(delay_dict, f)
#####################################

In [17]:
nx.write_gpickle(G, f"{netlist_name}.gpickle")

In [16]:
long = True
col = col_row[0]
row = col_row[1]
edgs = np.array([col, row]).T

if long:
    #####################################
    all_dist_raw = []
    edgs_d = [(lst[0], lst[1], 1) for lst in edgs] 
    n = len(np.unique(row + col))
    graph = Graph(edgs_d, n)
    for node in p_input: all_dist_raw.append(findLongestDistance(graph, node, n))

    # building the delay based dictionary
    delay_dict = all_dist_raw[0]
    delay_dict = {key: list(delay_dict[key]) for key in delay_dict.keys()}
    for dist in all_dist_raw[1:]:
        for key in dist.keys(): 
            val = dist[key]
            if key in delay_dict.keys():
                delay_dict[key] = delay_dict[key] + list(val)
            else:
                delay_dict[key] = list(val)

    delay_dict = {key: max(delay_dict[key]) for key in delay_dict.keys()}
    ##########################################
else:
    # building the delay based dictionary
    s_all_raw = []
    for v in p_input:
        s_all_raw.append(nx.shortest_path_length(G, v))    

    delay_dict = all_raw[0]
    delay_dict = {key: [delay_dict[key]] for key in delay_dict.keys()}
    for dist in s_all_raw[1:]:
        for key in dist.keys(): 
            val = dist[key]
            if key in delay_dict.keys():
                delay_dict[key] = delay_dict[key] + [val]
            else:
                delay_dict[key] = [val]

    delay_dict = {key: min(delay_dict[key]) for key in delay_dict.keys()}

In [17]:
with open('delay_dict.pkl', 'wb') as f:
    pickle.dump(delay_dict, f)

In [ ]:
nx.set_node_attributes(G, dall, "inst_name")
nx.set_node_attributes(G, dtype, "type")

In [ ]:
idx_d_dict = {}
for node in nodelist:
    try:
        inst = G.nodes[node]['inst_name']
        idx_d_dict[node] = float(new_d_dict[inst])
    except:
        continue

In [ ]:
s_inst = set()
for node in idx_d_dict.keys():
    delay = idx_d_dict[node]
    if np.isclose(delay, 0):
        s_inst.add(node)

In [ ]:
sub_delay = {node:delay_dict[node] for node in nodelist}

d_lst = []
for delay in range(max(sub_delay.values()) + 1):
    delay_level_lst = []
    for node in sub_delay.keys():
        if sub_delay[node] == delay:
            delay_level_lst.append(node)
    
    if len(delay_level_lst) == 0:
        continue
    d_lst.append(delay_level_lst)

In [ ]:
with open('sub_delay.pkl', 'wb') as f:
    pickle.dump(sub_delay, f)

In [ ]:
with open('sub_delay.pkl', 'rb') as f:
    sub_delay = pickle.load(f)

In [ ]:
mid = np.max([len(lst) for lst in d_lst])/2
pos = {}
for delay in range(len(d_lst)):
    lst = d_lst[delay]
    for in_pos in range(len(lst)):
        node = lst[in_pos]
        pos[node] = ((mid - len(lst)) + in_pos*2, (len(d_lst) - delay)*2)

In [ ]:
labels = {}
for lst in d_lst:
    for node in lst:
        if node in s_inst:
            val = str(delay_dict[node]) + " " + str(node)
            labels[node] = val

In [ ]:
f = plt.figure(figsize=(10,50), dpi=100) 
nx.draw(sub_G, node_size=20, pos=pos, width=0.3, arrowsize=5, node_color=color, labels=labels, font_size=8)
f.savefig("graph.png")

In [ ]:
n_nodelist = []
for lst in d_lst:
    for node in lst:
        n_nodelist.append(node)
    
nodelist = n_nodelist

In [ ]:
np.save("nodelist.npy", nodelist)

In [ ]:
w_d_lst = []
for idx1 in range(len(nodelist)):
    s_n = nodelist[idx1]
    dgms1 = np.load(f"dgms/pd_{s_n}.npy", allow_pickle=True).tolist()
    dgm1 = dgms1[0] + dgms1[1] + dgms1[2]
    dgm1 = np.array([tp[-1] for tp in dgm1])
    inner_lst = []
    for idx2 in range(idx1, len(nodelist)):
        dgms2 = np.load(f"dgms/pd_{nodelist[idx2]}.npy", allow_pickle=True).tolist()
        dgm2 = dgms2[0] + dgms2[1] + dgms2[2]
        dgm2 = np.array([tp[-1] for tp in dgm2])
        inner_lst.append(persim.sliced_wasserstein(dgm1, dgm2, M=50))
    
    w_d_lst.append(inner_lst)
    if idx1//500 > 0 and idx1%500 == 0:
        print(idx1)
        np.save("w_d_lst.npy", w_d_lst)
        
np.save("w_d_lst.npy", w_d_lst)

In [ ]:
sub_s = []
for node in nodelist:
    if node in s_inst:
        sub_s.append(node)

In [ ]:
w_d_lst = np.load("w_d_lst_1_nets.npy", allow_pickle=True)
N = len(w_d_lst)
b_symm = np.random.random(size=(N,N))
for idx in range(len(w_d_lst)):
    lst = w_d_lst[idx]
        val = lst[in_idx]
        b_symm[idx][(len(w_d_lst)-len(lst))+in_idx] = val
        b_symm[(len(w_d_lst)-len(lst))+in_idx][idx] = val
np.save("b_symm_0_nets.npy", b_symm)

In [ ]:
b_symm = np.load("b_symm_0_nets.npy")